In [ ]:
!pip install -q google-generativeai


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from getpass import getpass
import google.generativeai as genai

c:\Users\priya\Desktop\python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from getpass import getpass

if os.getenv("GEMINI_API_KEY") is None:
    os.environ["GEMINI_API_KEY"] = getpass("Please enter your Gemini API key: ")

genai.configure(api_key=os.environ["GEMINI_API_KEY"])


In [ ]:
from PIL import Image

prompt = """
Please output your analysis based on the format below:

Format examples:
Image shows: Several closed whiteheads appear on the forehead and lower cheeks, with no visible redness or swelling.
Diagnosis: Closed comedones
Severity: Mild
Features: Whiteheads, clustered distribution, no obvious inflammation

Image shows: Multiple inflamed red papules appear on both cheeks, with some pustules on the lesions.
Diagnosis: Inflammatory acne
Severity: Moderate
Features: Red papules, presence of pustules, localized redness

Please follow the format above:
Do not add extra commentary or explanations
Start directly with “Image shows: …”
Clearly label each section as Diagnosis / Severity / Features
"""



In [ ]:
image_folder = r"C:\Users\priya\Desktop\extra material\skinconditionproject\JPEGImages"
image_files = [
    "levle0_5.jpg",
    "levle2_20.jpg",
    "levle3_142.jpg"
]

for img_name in image_files:
    try:
        img_path = os.path.join(image_folder, img_name)
        img = Image.open(img_path)
        print(f"🔍 Analyzing: {img_name}")

        response = model.generate_content([img, prompt])
        print("🧠 Gemini replied:")
        print(response.text)
        print("-" * 60)

    except Exception as e:
        print(f"❌ Failed to analyze {img_name}: {e}")

🔍 Analyzing: levle0_5.jpg
❌ Failed to analyze levle0_5.jpg: name 'model' is not defined
🔍 Analyzing: levle2_20.jpg
❌ Failed to analyze levle2_20.jpg: name 'model' is not defined
🔍 Analyzing: levle3_142.jpg
❌ Failed to analyze levle3_142.jpg: name 'model' is not defined


In [10]:
import pandas as pd
db = pd.read_csv(r"C:\Users\priya\Desktop\extra material\skinconditionproject\skin_condition_knowledge_new.csv")
db.head()


,Condition,Description,Causes,Recommended Ingredients,Care Tips,When to Seek Help,Source URL
0,Cystic acne,"Large, red, painful breakouts deep in the skin.","Hormonal fluctuations (e.g., menstrual cycle, ...","Oral antibiotics, birth control pills, benzoyl...","Wash face twice a day, avoid scrubbing or popp...","If cysts are painful, recurrent, or cause scar...",https://www.webmd.com/skin-problems-and-treatm...
1,Blackheads,A type of acne that forms when dead skin cells...,"Typically caused by oil buildup, clogged folli...","Salicylic acid, Retinoid creams and lotions (v...","Wash face regularly, avoid harsh scrubbing, us...",If blackheads persist or worsen despite regula...,https://www.webmd.com/skin-problems-and-treatm...
2,Inflammatory acne,"Red, swollen, sore bumps containing pus, dead ...",Pores or follicles clogged with dead skin cell...,"Azelaic acid, Benzoyl peroxide, Niacinamide, R...",Avoid touching face. Use non-comedogenic produ...,If inflammation persists or worsens despite ro...,https://my.clevelandclinic.org/health/diseases...
3,Post-inflammatory hyperpigmentation (PIH),Temporary pigmentation following injury or inf...,Melanin deposition in keratinocytes due to epi...,"Hydroquinone, Azelaic acid, Cysteamine, Vitami...",Apply SPF 50+ sunscreen daily to exposed areas...,"If pigmentation worsens, persists, or causes e...",https://dermnetnz.org/topics/postinflammatory-...
4,Closed comedones,"Closed comedones, or whiteheads, are small, fl...","Excess oil production, buildup of dead skin ce...","Salicylic acid, Retinoids (tretinoin, adapalen...",Cleanse gently twice a day. Avoid oily skincar...,"If comedones persist, increase in number, or b...",Mayo Clinic / WebMD (compiled)


In [11]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Condition                9 non-null      object
 1   Description              9 non-null      object
 2   Causes                   9 non-null      object
 3   Recommended Ingredients  9 non-null      object
 4   Care Tips                9 non-null      object
 5   When to Seek Help        9 non-null      object
 6   Source URL               9 non-null      object
dtypes: object(7)
memory usage: 636.0+ bytes


In [ ]:
def get_skin_care_recommendation(diagnosis, df):
    """
    Search for recommended items in the knowledge base based on the Gemini diagnosis name.
    diagnosis: The diagnosis name output by the model (e.g., 'inflammatory acne')
    df: Knowledge base DataFrame
    Returns a dict; if not found, returns a prompt dictionary.
    """
    for _, row in df.iterrows():
        if diagnosis.lower() in row["Condition"].lower():
            return {
                "Condition": row["Condition"],
                "Description": row["Description"],
                "Causes": row["Causes"],
                "Recommended Ingredients": row["Recommended Ingredients"],
                "Care Tips": row["Care Tips"],
                "When to Seek Help": row["When to Seek Help"],
                "Source URL": row["Source URL"]
            }
    return {"Message": "No matching diagnosis found in knowledge base."}


In [ ]:
def generate_combined_response(diagnosis, df):
    rec = get_skin_care_recommendation(diagnosis, df)

    if "Message" in rec:
        return f"⚠️ Sorry, no guidance found for diagnosis: **{diagnosis}**."

    response = f"""📋 **Diagnosis Result**: {rec['Condition']}

🧾 **Condition Description**:
{rec['Description']}

🧪 **Likely Causes**:
{rec['Causes']}

💊 **Recommended Ingredients**:
{rec['Recommended Ingredients']}

🧼 **Daily Care Suggestions**:
{rec['Care Tips']}

🏥 **When to Seek Help**:
{rec['When to Seek Help']}

🔗 **Source**: {rec['Source URL']}
"""
    return response

In [ ]:
#test run

if __name__ == "__main__":
    test_diagnosis = "inflammatory acne"

    print("🔍 Raw result from knowledge base query:")
    raw_result = get_skin_care_recommendation(test_diagnosis, db)
    for key, value in raw_result.items():
        print(f"{key}: {value}")

    print("\n🤖 Chatbot response output:")
    chatbot_reply = generate_combined_response(test_diagnosis, db)
    print(chatbot_reply)

🔍 Raw result from knowledge base query:
Condition: Inflammatory acne
Description: Red, swollen, sore bumps containing pus, dead skin cells, and excess oil.
Causes: Pores or follicles clogged with dead skin cells, sebum, or oils. Ruptured pore walls spread inflammation deeper into the skin.
Recommended Ingredients: Azelaic acid, Benzoyl peroxide, Niacinamide, Retinoids (tretinoin, adapalene), Salicylic acid, Topical/oral antibiotics, Hormonal therapy, Isotretinoin.
Care Tips: Avoid touching face. Use non-comedogenic products. Don’t pop pimples. Eat a low-sugar diet. Wash and moisturize gently.
When to Seek Help: If inflammation persists or worsens despite routine skincare, or if nodules/cysts form.
Source URL: https://my.clevelandclinic.org/health/diseases/22765-inflammatory-acne

🤖 Chatbot response output:
📋 **Diagnosis Result**: Inflammatory acne

🧾 **Condition Description**:
Red, swollen, sore bumps containing pus, dead skin cells, and excess oil.

🧪 **Likely Causes**:
Pores or folli